# Deep Agent - Multi-Agent Deep Finance Researcher

Advanced financial research system using LangChain's DeepAgent with context isolation and strategic delegation.

## Overview

This notebook demonstrates:
- **DeepAgent Framework**: LangChain's advanced agent architecture
- **Context Isolation**: Sub-agents with isolated contexts for focused research
- **Strategic Delegation**: Orchestrator coordinates multiple specialist sub-agents
- **RAG + Live Data**: Hybrid search on SEC filings + Yahoo Finance integration
- **File-Based Memory**: Persistent research artifacts with secure sandbox (virtual_mode=True)

**Key Features:**
1. Orchestrator breaks down complex queries into research tasks
2. Sub-agents conduct focused research with isolated context
3. Think tool for strategic reflection and decision-making
4. Final report synthesis with consolidated citations
5. SQLite checkpointer for conversation memory
6. FilesystemBackend with virtual_mode for secure file operations

## Setup

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import os
import uuid
import sqlite3
from datetime import datetime
from IPython.display import Image, display

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.messages import HumanMessage, AIMessage, ToolMessage
from deepagents import create_deep_agent
from deepagents.backends import FilesystemBackend
from deepagents.backends.utils import file_data_to_string
from langgraph.checkpoint.sqlite import SqliteSaver

from scripts.rag_tools import hybrid_search, live_finance_researcher, think_tool
from scripts.deep_prompts import DEEP_RESEARCHER_INSTRUCTIONS, DEEP_ORCHESTRATOR_INSTRUCTIONS

## Initialize Memory and File Backend

In [4]:
# Secure filesystem backend for research outputs
RESEARCH_OUTPUT_DIR = os.path.join(os.getcwd(),"research_outputs")

In [5]:
def get_research_backend(thread_id):

    USER_OUTPUT_DIR = os.path.join(RESEARCH_OUTPUT_DIR, thread_id)
    
    os.makedirs(USER_OUTPUT_DIR, exist_ok=True)

    print(f"Writing research files to: {USER_OUTPUT_DIR}")

    # Create filesystem backend with virtual_mode=True for security
    backend = FilesystemBackend(
        root_dir=USER_OUTPUT_DIR,
        virtual_mode=True  # Prevents agent from accessing files outside sandbox
    )

    return backend

## Create Research Sub-Agent

In [6]:
# Get current date
current_date = datetime.now().strftime("%Y-%m-%d")

# Create research sub-agent with isolated context
research_sub_agent = {
    "name": "financial-research-agent",
    "description": "Delegate financial research to this sub-agent. Give it one specific research task at a time.",
    "system_prompt": DEEP_RESEARCHER_INSTRUCTIONS.format(date=current_date),
    "tools": [hybrid_search, live_finance_researcher, think_tool],
}

## Initialize LLM and Create DeepAgent

In [7]:
# Initialize model
model = ChatGoogleGenerativeAI(model='gemini-3-flash-preview')

# Tools for the main agent (orchestrator level)
tools = [hybrid_search, live_finance_researcher, think_tool]

In [8]:
def get_deep_agent(thread_id):

    # SQLite checkpointer for agent memory
    conn = sqlite3.connect('data/deep_agent_finance_researcher.db', check_same_thread=False)
    checkpointer = SqliteSaver(conn=conn)

    backend = get_research_backend(thread_id)

    # Create the deep agent with memory and secure file backend
    agent = create_deep_agent(
        model=model,
        tools=tools,
        system_prompt=DEEP_ORCHESTRATOR_INSTRUCTIONS,
        subagents=[research_sub_agent],
        checkpointer=checkpointer,  # SQLite memory
        backend=backend,  # Secure filesystem with virtual_mode=True
    )

    return agent

    # Visualize the agent graph
    agent

## Streaming Helper Function

In [9]:
def stream_deep_agent(agent, query, thread_id="default"):
    """Stream deep agent responses with tool call visibility."""
    
    config = {'configurable': {'thread_id': thread_id}}
    
    for chunk in agent.stream(
        {'messages': [HumanMessage(query)]},
        stream_mode='messages',
        config=config
    ):
        # Extract message from chunk
        message = chunk[0] if isinstance(chunk, tuple) else chunk
        
        # Handle AI messages with tool calls
        if isinstance(message, AIMessage) and message.tool_calls:
            for tool_call in message.tool_calls:
                print(f"\n  Tool Called: {tool_call['name']}")
                print(f"   Args: {tool_call['args']}")
                print()
        
        # Handle tool responses
        elif isinstance(message, ToolMessage):
            print(f"\n  Tool Result (length: {len(message.content)} chars)")
            print()
        
        # Handle AI text responses
        elif isinstance(message, AIMessage) and message.content:
            # Stream the text content
            print(message.content, end='', flush=True)

## Example 1: Simple Financial Query

In [10]:
from scripts.agent_utils import stream_agent_response

query = "What was Amazon's revenue in Q1 2024?"
thread_id="session1"

agent = get_deep_agent(thread_id)
stream_agent_response(agent, query, thread_id)

Writing research files to: d:\Courses\Udemy\Multi Agent Deep RAG\research_outputs\session1
Amazon's total consolidated revenue for the first quarter of 2024 was **$143.31 billion** ($143,313 million), which represents a **13% increase** compared to the $127.36 billion reported in Q1 2023 [1].

### Revenue Breakdown by Segment
The growth was driven by strong performance across all major business segments:
*   **North America:** $86.34 billion (12% year-over-year growth).
*   **International:** $31.94 billion (10% year-over-year growth).
*   **AWS (Amazon Web Services):** $25.04 billion (17% year-over-year growth) [1].

### Key Financial Metrics
*   **Consolidated YoY Growth:** 13% (consistent when excluding foreign exchange impacts).
*   **Operating Income:** Significantly increased to $15.3 billion in Q1 2024, compared with $4.8 billion in Q1 2023.
*   **Net Income:** Increased to $10.4 billion in Q1 2024, compared with $3.2 billion in Q1 2023.

### Sources
[1] Source file: Amazon Q1 2

In [ ]:
# Files are already displayed by stream_deep_agent helper
# The final report is printed automatically if available

In [12]:
query = "Compare Apple and Amazon's 2024 revenue and profitability. Present full and detailed report."
thread_id="session2"

agent = get_deep_agent(thread_id)
stream_agent_response(agent, query, thread_id)

Writing research files to: d:\Courses\Udemy\Multi Agent Deep RAG\research_outputs\session2

  Tool Called: write_todos
   Args: {'todos': [{'status': 'completed', 'content': 'Save research request to /research_request.md'}, {'status': 'in_progress', 'content': "Research Apple's 2024 revenue and profitability using a sub-agent"}, {'status': 'in_progress', 'content': "Research Amazon's 2024 revenue and profitability using a sub-agent"}, {'status': 'pending', 'content': 'Synthesize findings and write final report to /final_report.md'}, {'status': 'pending', 'content': 'Verify report completeness against research request'}]}


  Tool Called: task
   Args: {'description': "Research Apple's 2024 revenue and profitability. \nSpecifically look for:\n- Total revenue for fiscal year 2024 and any available calendar year 2024 data.\n- Gross margin, operating income, and net income for 2024.\n- Breakdown of revenue by segment (iPhone, Mac, iPad, Services, etc.) if available for the full year.\n- Ye

In [ ]:
# Files are already displayed by stream_deep_agent helper
# The final report is printed automatically if available

In [ ]:
query = "Compare Apple and Google's 2024 revenue and profitability"
thread_id="session3"

agent = get_deep_agent(thread_id)
stream_agent_response(agent, query, thread_id)

## Example 4: Historical + Live Data

In [ ]:
query = "Research Meta's 2024 annual financial performance from SEC filings and compare it with current stock performance"
thread_id="session4"

agent = get_deep_agent(thread_id)
stream_agent_response(agent, query, thread_id)